In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/MyDrive')

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
import glob
import cv2

# Set image size and latent dimension
IMAGE_SIZE = 512  # 512x512 images for real PAN cards
LATENT_DIM = 100  # Latent vector size
BATCH_SIZE = 32
EPOCHS = 10

# Path to your dataset directory
dataset_dir = 'pre_processed_pan_cards/train/real'

# Load images
def load_images(image_size=IMAGE_SIZE):
    images = []
    for img_path in glob.glob(os.path.join(dataset_dir, '*.jpg')):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (image_size, image_size))  # Resize image
        images.append(img)
    images = np.array(images, dtype='float32')
    # Normalize images to [-1, 1]
    images = (images - 127.5) / 127.5
    return images

# Load your dataset
dataset = load_images()

# Build the Generator Model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(4 * 4 * 1024, use_bias=False, input_shape=(LATENT_DIM,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((4, 4, 1024)),
        layers.Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # Output layer with 512x512x3 (RGB channels)
        layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # Ensure output size is 512x512x3 (RGB)
        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')  # 512x512x3 output
    ])
    return model

# Build the Discriminator Model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)  # Output a single score (real or fake)
    ])
    return model

# Define the GAN Model
def build_gan(generator, discriminator):
    discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False  # During GAN training, freeze the discriminator

    gan_input = layers.Input(shape=(LATENT_DIM,))
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan = models.Model(gan_input, gan_output)
    gan.compile(optimizer='adam', loss='binary_crossentropy')
    return gan

# Train step function
def train_step(real_images, latent_dim, generator, discriminator, gan):
    noise = np.random.normal(0, 1, (BATCH_SIZE, LATENT_DIM))  # Generate random noise
    generated_images = generator(noise, training=True)

    # Labels for real and fake images
    real_labels = np.ones((BATCH_SIZE, 1))
    fake_labels = np.zeros((BATCH_SIZE, 1))

    # Train the discriminator
    with tf.GradientTape() as tape:
        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(generated_images, training=True)
        real_loss = tf.keras.losses.binary_crossentropy(real_labels, real_output)
        fake_loss = tf.keras.losses.binary_crossentropy(fake_labels, fake_output)
        d_loss = real_loss + fake_loss

    d_gradients = tape.gradient(d_loss, discriminator.trainable_variables)
    discriminator.optimizer.apply_gradients(zip(d_gradients, discriminator.trainable_variables))

    # Train the generator via GAN model (discriminator is frozen)
    with tf.GradientTape() as tape:
        gan_loss = gan(noise, training=True)

    g_gradients = tape.gradient(gan_loss, generator.trainable_variables)
    generator.optimizer.apply_gradients(zip(g_gradients, generator.trainable_variables))

    return d_loss, gan_loss

# Train the models
def train(dataset, epochs):
    generator = build_generator()
    discriminator = build_discriminator()
    gan = build_gan(generator, discriminator)

    discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

    # Training loop
    for epoch in range(epochs):
        for batch_start in range(0, dataset.shape[0], BATCH_SIZE):
            real_images = dataset[batch_start:batch_start+BATCH_SIZE]
            d_loss, g_loss = train_step(real_images, LATENT_DIM, generator, discriminator, gan)

        print(f'Epoch {epoch+1}/{epochs}, D Loss: {d_loss.numpy()}, G Loss: {g_loss.numpy()}')

# Start training
train(dataset, EPOCHS)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "functional_17" is incompatible with the layer: expected shape=(None, 512, 512, 3), found shape=(None, 256, 256, 3)[0m

Arguments received by Sequential.call():
  • args=('<KerasTensor shape=(None, 256, 256, 3), dtype=float32, sparse=False, name=keras_tensor_326>',)
  • kwargs={'mask': 'None'}

In [2]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: /content
